 # Drop Rate in Fishing and Foraging DFK Quests
 Analysis and Visualization 
 

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots

if not os.path.exists('imgs'):
    os.makedirs('imgs')

IMG_SIZE = [1000,700]

In [3]:
def plot2d(df,cols3, t, y, decimals=2, img_size = [700,500], img_name = None):
    df = pd.pivot_table(df, values=cols3[0], index=cols3[1], columns=[cols3[2]])

    fig = ff.create_annotated_heatmap(
                z=df.to_numpy().round(decimals=decimals),
                x=df.columns.tolist(),
                y=df.index.tolist(),
                colorscale=['red', 'orange', 'yellow', 'green'],
                hoverongaps=True
                )

    r = fig.update_layout(title_text=f'<i><b>{t}</b></i>',
                      yaxis = dict(title=y),
                      xaxis = dict(title='Profession Level')
                     )
    r =fig['layout']['xaxis']['side'] = 'bottom'
    r = fig.update_layout(width = img_size[0], height = img_size[1], margin=dict(t=50, l=100))
    fig['data'][0]['showscale'] = True
    fig.show()
    if img_name:
        fig.write_image(img_name)

In [4]:
def df_diff(df1,df2,col,min_cnt):
    d1 = df1.groupby(['stats','level'])[col].agg(['mean','count']).reset_index()
    d2 = df2.groupby(['stats','level'])[col].agg(['mean','count']).reset_index()
    d = pd.merge(d1,d2,on=['stats','level'])
    d[col] = d['mean_x']-d['mean_y']
    return d[(d['count_x']>min_cnt) &(d['count_y']>min_cnt)]

In [5]:
usecols = ['level', 'stats', 'stamina', 'DFKGOLD', 'DFKTEARS', 'DFKSHVAS', 'DFKEGG']
fo = pd.read_csv('./data/foraging.csv', usecols = usecols)
fi = pd.read_csv('./data/fishing.csv', usecols = usecols)

In [6]:
fo
fi

,level,stats,stamina,DFKGOLD,DFKTEARS,DFKSHVAS,DFKEGG
0,0,17,5,2.5,0,0,0
1,0,17,5,0.0,0,0,0
2,0,17,5,0.0,0,0,0
3,0,17,5,2.5,0,0,0
4,0,15,5,5.0,0,0,0
...,...,...,...,...,...,...,...
4002954,10,30,5,0.0,0,0,0
4002955,14,27,5,0.0,0,0,0
4002956,14,27,5,0.0,1,0,0
4002957,10,30,5,2.5,0,0,0


,level,stats,stamina,DFKGOLD,DFKTEARS,DFKSHVAS,DFKEGG
0,0,14,7,2.5,0,0,0
1,0,21,7,15.0,0,0,0
2,0,21,7,5.0,0,0,0
3,0,21,7,15.0,0,0,0
4,0,20,7,5.0,0,0,0
...,...,...,...,...,...,...,...
4041616,11,17,5,0.0,0,0,0
4041617,11,17,5,2.5,0,0,0
4041618,11,17,5,2.5,0,0,0
4041619,11,17,5,10.0,1,0,0


# DFKGOLD

In [ ]:
for col in ['DFKGOLD']:
    min_cnt = 100
  
    df = fi[fi['stamina'] == 5].groupby(['stats','level'])[col].agg(['mean','count']).reset_index()
    df = df[df['level']<=15]
    plot2d(df[df['count']>min_cnt], ['mean','stats','level'], f'<b>FISHING</b>: Average {col} per Quest for Fishers','Stats (AGI+LCK)', decimals = 1, img_size = IMG_SIZE, img_name=f'./imgs/{col}_2d_fishing_fishers.png')
  
    df = fi[fi['stamina'] == 7].groupby(['stats','level'])[col].agg(['mean','count']).reset_index()
    df = df[df['level']<=15]
    plot2d(df[df['count']>min_cnt], ['mean','stats','level'], f'<b>FISHING</b>: Average {col} per Quest for Non Fishers','Stats (AGI+LCK)', decimals = 1, img_size = IMG_SIZE, img_name=f'./imgs/{col}_2d_fishing_others.png')   

    df = df_diff(fi[fi['stamina'] == 5],fi[fi['stamina'] == 7],col,min_cnt)
    df = df[df['level']<=15]
    plot2d(df,['DFKGOLD','stats','level'],f'FISHING: Fishers vs Non-Fishers: Difference in Average {col} per Quest','stats (AGI+LCK)', decimals = 1, img_size = IMG_SIZE, img_name=f'./imgs/{col}_2d_fishers_vs_non_fishers.png')
    print(f"Fishers earn an average + {df[col].mean():.2} gold per quest in recpect to Non-Fishers")

    df = fo[fo['stamina'] == 5].groupby(['stats','level'])[col].agg(['mean','count']).reset_index()
    df = df[df['level']<=15]
    plot2d(df[df['count']>min_cnt], ['mean','stats','level'], f'<b>FORAGING</b>: Average {col} per Quest for Foragers','Stats (DEX+INT)', decimals = 1, img_size = IMG_SIZE, img_name=f'./imgs/{col}_2d_foraging_foragers.png')
     
    df = fo[fo['stamina'] == 7].groupby(['stats','level'])[col].agg(['mean','count']).reset_index()
    df = df[df['level']<=15]
    plot2d(df[df['count']>min_cnt], ['mean','stats','level'], f'<b>FORAGING</b>: Average {col} per Quest for Non Foragers','Stats (DEX+INT)', decimals = 1, img_size = IMG_SIZE, img_name=f'./imgs/{col}_2d_foraging_others.png')

    df = df_diff(fo[fo['stamina'] == 5],fo[fo['stamina'] == 7],col,min_cnt)
    df = df[df['level']<=15]
    plot2d(df,['DFKGOLD','stats','level'],f'FORAGING: Foragers vs Non-Foragers: Difference in Average {col} per Quest','stats (DEX+INT)', decimals = 1, img_size = IMG_SIZE, img_name=f'./imgs/{col}_2d_foragers_vs_non_foragers.png')
    print(f"Foragers earn an average + {df[col].mean():.2} gold per quest in recpect to Non-Foragers")
    
    
    min_cnt = 100
    df = df_diff(fo[(fo['stamina'] == 5)],fi[fi['stamina'] == 5],col,min_cnt)
    df = df[df['level']<=15]
    plot2d(df,['DFKGOLD','stats','level'],f'FORAGING Foragers vs FISHING Fishers: Difference in Average {col} per Quest','stats (DEX+INT for Foragers, AGI+LCK for Fishers)', decimals = 1, img_size = IMG_SIZE, img_name=f'./imgs/{col}_2d_fishing_vs_foraging.png')
    print(f"Foragers earn an average + {df[col].mean():.2} gold per quest in recpect to Fishers")


# Gaia's Tears, Shiva Runes, and Eggs

In [ ]:
def calc_means(d, main_prof, agg ):
    d['main_prof'] = d['stamina'].apply(lambda x: main_prof if x ==5 else 'other')
    d1 = d.groupby(['main_prof',agg])['DFKGOLD','DFKTEARS','DFKSHVAS','DFKEGG'].agg(['mean','std']).reset_index()
    d1.columns = [col[0] if (col[1] == '' or col[1] == 'mean') else '_'.join((col[0], str(col[1])))  for col in d1.columns]
    d2 = d.groupby(['main_prof',agg]).agg(count = pd.NamedAgg(column='DFKGOLD',aggfunc='count'))
    return pd.merge(d1,d2,on=[agg,'main_prof'])

def plot(dfs,column,xaxis, title="T", img_size = [700,500], img_name = None):
    fig = make_subplots(specs=[[{"secondary_y": False}]])
    profession = ['foraging','fishing']
    colors=['red','blue']
    line_dash = ['dot',None]
    i = 0
    j = 0
    for df in dfs:
        for x, tmp in df.groupby('main_prof'):
            r = fig.add_trace(
                go.Scatter(x=tmp[xaxis], y=tmp[column], name=f"{profession[i]}_{x}", line=dict(color=colors[i], dash=line_dash[(j+1)%2])),
                secondary_y=False,
            )
            j+=1
        i+=1

    r = fig.update_layout(title_text=f"<b>{title}</b>")
    r = fig.update_xaxes(title_text=xaxis)
    r = fig.update_yaxes(title_text='Drop per Quest')
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0
    ))
    r = fig.update_layout(width = img_size[0], height = img_size[1])
    fig.show()
    if img_name:
        fig.write_image(img_name)

In [ ]:
fos = calc_means(fo, 'forager','stats')
fos = fos[fos['stats']<=25]
fis = calc_means(fi, 'fisher','stats')
fis = fis[fis['stats']<=25]

fol = calc_means(fo,'forager','level')
fol = fol[fol['level']<=8]
fil = calc_means(fi,'fisher','level')
fil = fil[fil['level']<=8]

In [ ]:
for col,decimal in zip(['DFKTEARS','DFKSHVAS','DFKEGG'],[3,3,4]):
    min_cnt = 1000
    plot([fos,fis], col, 'stats', title=col, img_name = f"./imgs/{col}_1d_stats.png")
    plot([fol,fil], col, 'level', title=col, img_name = f"./imgs/{col}_1d_level.png")
#     df = fo[fo['stamina'] == 5].groupby(['stats','level'])[col].agg(['mean','count']).reset_index()
#     plot2d(df[df['count']>min_cnt] ,['mean','stats','level'],f'Foragers Average {col} per Quest','Stats (DEX+INT)',decimal, img_size = IMG_SIZE, img_name = f"./imgs/{col}_2d_foragers.png")
#     df = fi[fi['stamina'] == 5].groupby(['stats','level'])[col].agg(['mean','count']).reset_index()
#     plot2d(df[df['count']>min_cnt] ,['mean','stats','level'],f'Fishers Average {col} per Quest','Stats (AGI+LCK)',decimal, img_size = IMG_SIZE, img_name = f"./imgs/{col}_2d_foragers.png")